In [1]:
from Polygon import Vertex, Polygon
from Fiber import Edge,Fiber
from Configuration import plot_configuration, initialize_a_fiber,initialize_a_polygon
from FIREminimization import calculateForceVelocityProjections, FIREminimize
import os
import numpy as np

In [2]:
os.system('rm -rf output/ && mkdir output')

# testing FIRE minimization
polygon = initialize_a_polygon(P0 = 3.9)
fiber = initialize_a_fiber()



print("Initial area: ", polygon.area_)
print("Initial perimeter: ", polygon.perimeter_)
print("initial stress:\n", polygon.stress_)
print("initial edge lengths: ", [edge.length_ for edge in fiber.edges_] )
FIREminimize(polygon,fiber)

# P.compute_forces()
# P.compute_stress()

print("Minimized perimeter ",polygon.perimeter_)
print("Minimized area ",polygon.area_)
print("Minimized stress:\n",polygon.stress_)
print("Minimized edge lengths: ", [edge.length_ for edge in fiber.edges_] )

print("Forces after minimization:")
print("     Fixed polygon vertex: ",polygon.vertices_[0].force_)
print("     Fixed fiber node: ",fiber.edges_[0].nodes_[0].force_)
print("     Dot product: ",np.dot(polygon.vertices_[0].force_,fiber.edges_[0].nodes_[1].force_))


Initial area:  0.9880829342591236
Initial perimeter:  3.6305808079289723
initial stress:
 [[-0.76722073 -0.0085715 ]
 [-0.0085715  -0.77248759]]
initial edge lengths:  [0.4832448913802037, 0.4619923062597339]
Iteration  0 ff =  245.1119660793069 vv =  2.4632934827930215e-06 power =  0.024551776014597196
     Vertex  [5.61226551 9.48843204]
     Vertex  [-0.17289956 -0.14252292]
     Vertex  [-0.05325829 -0.38491343]
     Vertex  [ 0.23752461 -0.25120367]
     Vertex  [0.33765324 0.0056386 ]
     Vertex  [0.23946508 0.29728947]
     Vertex  [-0.04410226  0.29539215]
     Vertex  [-0.17657014  0.17056488]
     Node  [-5.98338343 -9.31115863]
     Node  [-0.36395253 -0.26539704]
     Node  [-0.36395253 -0.26539704]
     Node  [0.36725776 0.09787856]
Iteration  10000 ff =  9.630497518773972e-16 vv =  4.8272446580109194e-14 power =  6.816603742134358e-15
     Vertex  [ 2.37920285e-09 -3.82592269e-09]
     Vertex  [-9.87652690e-08 -9.35221652e-08]
     Vertex  [1.06413312e-08 6.47340263e-09]